<a href="https://colab.research.google.com/github/LeandroCoelhos/data_hackers/blob/main/Mapa_dos_Skills_na_Carreira_de_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quais os skills essenciais para profissionais de dados?

O mercado de dados está cada vez mais competitivo, com alta demanda por talentos capacitados. Isto, aliado aos inúmeros cursos prometendo empregabilidade rápida, exige dos profissionais e interessados em ingressar na área uma busca incessante de maneiras de impulsionar suas carreiras.

Como entender as habilidades cruciais para os profissionais de dados e como eles ingressaram nesse campo? Para responder a essa pergunta, conduzi este estudo com base na pesquisa **State of Data 2022** realizada com profissionais da área de dados.

O objetivo é fornecer insights  práticos para que os leitores conquistem uma vantagem ao compreender **as habilidades mais valorizadas** pelos empregadores e setores da indústria.

## Principais insights:

1)    O mercado de dados está em expansão, o que aumenta a demanda por profissionais qualificados e com habilidades específicas.

2)    As tecnologias relacionadas a fonte de dados, como SQL e Planilhas, são essenciais e amplamente requisitadas em diferentes setores. Por outro lado existe diversidade quanto a quais banco de dados são utilizados.

3)   Alta utilização de SQL e Python entre os profissionais de dados como principal linguagem de programação.

4) Profissionais com nivel de formação superior são a maioria na área.

5)    A busca contínua por aprendizado e desenvolvimento é fundamental para se manter relevante em um mercado em constante evolução, garantindo oportunidades de crescimento profissional e contribuição para o sucesso das empresas.


In [13]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from itertools import combinations
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
import networkx as nx

### Dados de entrada pesquisas
SoD22 = pd.read_csv('https://raw.githubusercontent.com/LeandroCoelhos/data_hackers/main/data/State_of_data_2022.csv')
SoD21 = pd.read_csv('https://raw.githubusercontent.com/LeandroCoelhos/data_hackers/main/data/State%20of%20Data%202021%20-%20Dataset%20-%20Pgina1.csv', low_memory=False)
# SoD19 = pd.read_csv('/kaggle/input/pesquisa-data-hackers-2019/datahackers-survey-2019-anonymous-responses.csv', low_memory=False) -- não tem info de atuação dos profissionais

num_amostra_inicial = len(SoD22)

# Criando um dicionario para facilitar a renomeação das colunas nos graficos
column_renaming = {col: col.split(", ")[1][1:-1].replace("'",'').strip() for col in SoD22.columns}

# facilitar selecao das colunas
def get_list_columns(prefix, df):
    return [col for col in df.columns if col.startswith(f"('{prefix}")]

Serão considerados nesse estudo apenas as respostas dos profissionais que estão atuando no Brasil, buscando mapear o mercado de dados no país.

Inicialmente, deseja-se realizar o mapeamento das áreas de atuação e os setores da indústria envolvidos. Na sequência, as habilidades comuns à áreas de destaque serão exploradas.


In [14]:
# filtro para profissionais que residem no país.
SoD22 = SoD22[SoD22["('P1_g ', 'vive_no_brasil')"]==True]
# filtro para profissionais empregados
SoD22 = SoD22[~SoD22["('P2_a ', 'Qual sua situação atual de trabalho?')"].isin(['Desempregado, buscando recolocação',
                                 'Prefiro não informar',
                                 'Somente Estudante (pós-graduação)',
                                 'Somente Estudante (graduação)',
                                 'Vivo no Brasil e trabalho remoto para empresa de fora do Brasil',
                                 'Desempregado e não estou buscando recolocação'])]
num_amostra_filtrada = len(SoD22)

## Situação atual dos dados da pesquisa
Antes de mergulharmos nas habilidades e níveis de conhecimento dos profissionais de dados, é essencial compreender a atual situação dos dados da pesquisa.

In [15]:
print("Total de profissionais na pesquisa: ", num_amostra_inicial)
print("Total de profissionais atuando no mercado nacional: ",num_amostra_filtrada)
percentual_amostra = round(num_amostra_filtrada/num_amostra_inicial,4 )*100
print("Profissionais atuando no mercado nacional correspondem a "
      + "\033[1m" + f"{percentual_amostra}%"+ "\033[0m"+
      " do total de participantes da pesquisa.")

Total de profissionais na pesquisa:  4271
Total de profissionais atuando no mercado nacional:  3607
Profissionais atuando no mercado nacional correspondem a 84.45% do total de participantes da pesquisa.


Temos muitos dados!! A pesquisa teve um alto engajamento de profissionais. Com essa amostra sólida, vamos conseguir explorar o cenário nacional da área de dados.

Agora vamos entender quais como as áreas de atuação estão distribuidas entre os respondentes.

# 1. O cenário atual do mercado de dados

Primeiramente, iremos analisar como os profissionais estão distribuídos nas diferentes áreas de atuação. O gráfico a seguir apresenta as principais áreas abordadas na pesquisa e o número de respondentes em cada atuação.

In [16]:
color_top = ["#460073", "#7500c0", "#dcafff","#a100ff"]
color_default = "#96968c"
opacity_top = 1
opacity_default = .25

In [17]:
def calc_percentual(num_profissionais, num_amostra_filtrada):
    return round(num_profissionais / num_amostra_filtrada, 4) * 100

def rename_areas(area):
    if area == 'Outra':
        return 'Outra atuação'
    elif area == 'Buscando emprego na área de dados.':
        return 'Buscando oportunidade na área de dados'
    return area

# Tratar dados de 2022
df_areas22 = SoD22[ "('P4_a_1 ', 'Atuacao')"].value_counts().reset_index()
df_areas22 = df_areas22.rename(columns={"index": "Area", "('P4_a_1 ', 'Atuacao')": "NumProfissionais"})
df_areas22["percentual"] = df_areas22["NumProfissionais"].apply(lambda x: calc_percentual(x, num_amostra_filtrada))
df_areas22["ano"] = 2022
df_areas22 = df_areas22.sort_values("percentual", ascending=False)
df_areas22["percentCumul"] = df_areas22["percentual"].cumsum()
df_areas22 = df_areas22.sort_values('NumProfissionais', ascending=True)

# Obter dados de 2021 para comparação
df_areas21 = SoD21[ "('P4_a ', 'Atuacao')"].value_counts().reset_index()
df_areas21 = df_areas21.rename(columns={"index": "Area", "('P4_a ', 'Atuacao')": "NumProfissionais"})
df_areas21['Area'] = df_areas21['Area'].apply(rename_areas)
df_areas21["percentual"] = df_areas21["NumProfissionais"].apply(lambda x: calc_percentual(x, num_amostra_filtrada))
df_areas21["ano"] = 2021
df_areas21 = df_areas21.sort_values("percentual", ascending=True)

# Unir dfs e calcular delta entre anos
df = pd.merge(df_areas22, df_areas21, on="Area", how="left")
df['delta'] = (df["NumProfissionais_x"] - df["NumProfissionais_y"]) * 100 / df["NumProfissionais_y"]
df_areas22 = pd.merge(df_areas22, df[["Area", "delta"]], on="Area", how='left')

In [18]:
def create_dumbbell_year_growth(df_areas21, df_areas22, color_default, color_top, opacity_default, opacity_top):
    fig = make_subplots(rows=1, cols=1, horizontal_spacing=0)

    # Adicionar os dados de 2022 com cor destaque
    fig.add_trace(go.Scatter(
        x=df_areas22['NumProfissionais'],
        y=df_areas22['Area'],
        name=str(df_areas22['ano'][0]),
        hoverinfo='text',
        hovertext=[f"Area: {cargo}<br>Numero: {num:.2f}<br>Ano: 2021<br>Percentual: {per:.2f}%<br>Delta: {delta:.2f}%"
                   for cargo, num, per, delta in zip(df_areas22['Area'],
                                                     df_areas22['NumProfissionais'],
                                                     df_areas22['percentual'],
                                                     df_areas22['delta'])],
        hoverlabel_font_color="white",
        mode='markers',
        marker=dict(
            color=color_top[1],
            line_color='white',
            line_width=.5,
            size=12,
            opacity=opacity_top
        ),
        legendgroup='2022',
        showlegend=True),
        row=1,
        col=1)

    # Adicionar os dados de 2021 com cor default
    fig.add_trace(go.Scatter(
        x=df_areas21['NumProfissionais'],
        y=df_areas21['Area'],
        name=str(df_areas21['ano'][0]),
        hoverinfo='text',
        hovertext=[f"Area: {cargo}<br>Numero: {num:.2f}<br>Ano: 2021<br>Percentual: {per:.2f}%"
                   for cargo, num, per in zip(df_areas21['Area'], df_areas21['NumProfissionais'], df_areas21['percentual'])],
        hoverlabel_font_color="white",
        mode='markers',
        marker=dict(
            symbol='square',
            color=color_default,
            line_color='white',
            line_width=.5,
            size=12,
            opacity=opacity_default
        ),
        legendgroup='2021',
        showlegend=True),
        row=1,
        col=1)

    # Criar indicador de variação = "setinha"
    for i in range(len(df_areas22)):
        fig.add_trace(go.Scatter(
            x=[df_areas21['NumProfissionais'].iloc[i], df_areas22['NumProfissionais'].iloc[i]],
            y=[df_areas22['Area'].iloc[i], df_areas22['Area'].iloc[i]],
            mode="markers+lines",
            hoverinfo='skip',
            marker=dict(
                symbol="arrow",
                color=color_default,
                opacity=opacity_default,
                size=12,
                angleref="previous",
                standoff=8
            ),
            line=dict(color="#dadada", width=1.5),  # Use grey dashed line for arrows
            legendgroup='2021',
            showlegend=False),
        row=1,
        col=1)

    #  anotações para os principais insights
    annotations = [
        go.layout.Annotation(
            xref='x', yref='y',
            x=df_areas22.loc[df_areas22['NumProfissionais'].idxmax(), 'NumProfissionais'],
            y=df_areas22.loc[df_areas22['NumProfissionais'].idxmax(), 'Area'],
            text='Análise conta com <br> maior número',
            yshift=10,
            font=dict(size=12, color='white'),
            bordercolor=color_top[3],
            borderwidth=2,
            borderpad=4,
            bgcolor=color_top[3],
            opacity=0.7
        ),
        go.layout.Annotation(
            xref='x', yref='y',
            x=df_areas22.loc[df_areas22['delta'].idxmax(), 'NumProfissionais'],
            y=df_areas22.loc[df_areas22['delta'].idxmax(), 'Area'],
            text='Engenharia conta com <br> maior crescimento',
            xshift=5,
            yshift=-10,
            ax=50,
            font=dict(size=12, color='white'),
            ayref="y",
            bordercolor=color_top[3],
            borderwidth=2,
            borderpad=4,
            bgcolor=color_top[3],
            opacity=0.7
        ),
    ]

    # Atualizar layout da figura
    fig.update_layout(
        annotations=annotations,
        title_text='Análises lidera mas Engenharia tem maior ascensão',
        title={
            'x': 0.5,
            'xanchor': 'center'
        },
        font_color="#42293f",
        font_size=14,
        font_family = "Graphic",
        xaxis=dict(
            showgrid=True,
            gridcolor='#e6e6dc',
            gridwidth=0.05,
            tickfont_color="#42293f",
            showticklabels=True,
            dtick=250,
            ticks='outside',
            tickcolor="#42293f"
        ),
        yaxis=dict(showgrid=False),
        xaxis2=dict(showticklabels=True),
        yaxis2=dict(
            side='right',
            mirror="allticks",
            showticklabels=False
        ),
        width=800,
        height=400,
        paper_bgcolor='white',
        plot_bgcolor='white',
        hovermode='closest',
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
    )

    fig.show()



create_dumbbell_year_growth(df_areas21, df_areas22, color_default, color_top, opacity_default, opacity_top)


De 2021 para 2022,podemos interpretar o aumento no número de profissionais em cada área de duas maneiras: a área de dados está em crescimento ou houve um maior engajamento na pesquisa, resultando em mais participantes. Provavelmente, essa tendência seja uma combinação dos dois fatores.

Além disso podemos perceber os seguintes pontos:
*    **Análise de dados é a área de dados mais popular**. Em 2022, havia mais analistas de dados do que qualquer outra área de dados.
*    **Engenharia de dados é a área de dados com maior crescimento**. De 2021 para 2022, o número de cientistas de dados aumentou em 57%.
* Em 2022, o número de profissionais de dados em busca de oportunidades reduziu em comparação com o ano anterior. Isso pode indicar que muitos desses profissionais encontraram a recolocação que desejavam.

Através da amostra de respondentes da pesquisa, observamos predominância significativa de **analistas, cientistas e engenheiros de dados**. Estes grupos serão o **público foco do nosso estudo**, assim como foi foco das perguntas da pesquisa. A partir de agora iremos observar apenas os resultados destes grupos que temos interesse em mapear as habilidades técnicas.

### Presença de Profissionais de Dados em Diferentes Setores Industriais
Agora, iremos analisar a distribuição desses profissionais nos diversos setores do mercado que utilizam dados em suas áreas de atuação. Vamos entender em quais setores esses profissionais estão atuando e identificar o setor com maior presença de profissionais de dados. O gráfico a seguir apresenta o **fluxo de profissionais de dados para os diferentes setores** nos quais eles atuam.


In [29]:
SoD22.groupby([ "('P2_b ', 'Setor')"], as_index=False).agg({ "('P0', 'id')": "count", "('P1_a ', 'Idade')":"mean"})

,"('P2_b ', 'Setor')","('P0', 'id')","('P1_a ', 'Idade')"
0,Agronegócios,52,30.470588
1,Educação,86,30.917647
2,Entretenimento ou Esportes,23,30.608696
3,Filantropia/ONG's,8,27.750000
4,Finanças ou Bancos,436,29.211009
5,Indústria,153,31.582781
6,Internet/Ecommerce,119,28.798319
7,Marketing,61,27.786885
8,Outra Opção,160,29.675000
9,Seguros ou Previdência,38,28.540541


In [52]:
# Filtro para manter apenas as atuações de foco
atuacoes_foco = ['Engenharia de Dados', 'Ciência de Dados', 'Análise de Dados']
SoD22 = SoD22[SoD22["('P4_a_1 ', 'Atuacao')"].isin(atuacoes_foco)]

# df com o total por setor
df_setor_all = (SoD22.groupby([ "('P2_b ', 'Setor')"], as_index=False)
                 .agg({ "('P0', 'id')": "count" })
                 .rename(columns={ "('P2_b ', 'Setor')": "Setor", "('P0', 'id')": "TotalSetor" })
               )
total = df_setor_all.TotalSetor.sum()
df_setor_all['percentual'] = df_setor_all.TotalSetor.apply(lambda x: (x / total) * 100)

#gambiarra para deixar outra opcao por ultimo pq tem um valor alto e não to a fim
df_setor_all = df_setor_all[df_setor_all.Setor != "Outra Opção"].sort_values("TotalSetor", ascending=False)
df_setor_all = pd.concat([df_setor_all, df_setor_all[df_setor_all.Setor == "Outra Opção"]])

df_setor_all['percentual_cumul'] = df_setor_all.percentual.cumsum()

# df por atuacao
df_setor22 = (SoD22.groupby([ "('P4_a_1 ', 'Atuacao')", "('P2_b ', 'Setor')"], as_index=False)
                .agg({ "('P0', 'id')": "count" })
                .rename(columns=column_renaming)
             )
df_setor22 = df_setor22.merge(df_setor_all, on="Setor", how="left")
df_setor22 = df_setor22[df_setor22['percentual_cumul'] < 85] # filtro para minimizar setores que aparecem -- tomando os 85% mais com maior demanda
setores_foco = df_setor22[df_setor22['percentual_cumul'] < 85].Setor.unique()
df_setor22 = df_setor22.sort_values("TotalSetor", ascending=False)

# cores para setores
df_setor22['color'] = "#dadada"
df_setor22.loc[df_setor22["Setor"] == "Tecnologia/Fábrica de Software", 'color'] = color_top[3]
df_setor22.loc[df_setor22["Setor"] == "Finanças ou Bancos", 'color'] = color_top[2]

# Definindo nodes e fluxos (atuação->setor)
label_from = list(df_setor22.Atuacao.unique())
label_to = list(df_setor22.Setor.unique())
labels = label_from + label_to
df_setor22 = df_setor22.sort_values("id")
df_setor22['from'] = df_setor22.Atuacao.apply(lambda x: labels.index(x, 0, len(labels)))
df_setor22['to'] = df_setor22.Setor.apply(lambda x: labels.index(x, 0, len(labels)))


In [53]:
def create_sankey_flow(df_setor22):
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=5,
            thickness=15,
            line=dict(color="#dadada", width=0),
            label=[f'<b>{x}</b>' for x in labels],
            color="#460073",
            customdata=labels,
            hovertemplate='%{customdata}<br>Total profissionais: %{value}<extra></extra>',
        ),
        link=dict(
            source=df_setor22['from'],
            target=df_setor22['to'],
            value=df_setor22.id,
            color=df_setor22['color'],
            label=df_setor22['to'],
            hovertemplate='Atuação: %{source.customdata}<br />'+
                           'Setor: %{target.customdata}<br />#Profissionais: %{value}'+
                           '<extra></extra>',
        )
    )])

    # Configurar layout do gráfico
    fig.update_layout(
        title_text="Fábrica de Software e Finanças na frente da demanda por profissionais",
        title={
            'x': 0.5,
            'xanchor': 'center'
        },
        font_size=14,
        height=600,
        width=800,
        font_color="#42293f",
        font_family = "Graphic"
    )

    # Mostrar o gráfico
    fig.show()

create_sankey_flow(df_setor22)

Pode-se perceber que as áreas que tem mais demanda por estes profissionais sao **tecnologia/fábrica de software e finanças ou bancos,** seguido pela **área de consultorias** e pelo **setor de varejo**. Surpreendentemente estes 4 setores são responsáveis por 53,5% da demanda por profissionais de dados. As 13 áreas indicadas representam 85% da demanda, o que reforça sua importância no mercado. A partir de agora, iremos focar nossa análise nessas 13 áreas-chave para compreender melhor as competências essenciais necessárias para prosperar nesses setores.

### Identificando setores semelhantes com base em competências de dados

Uma pergunta interessante a se fazer é qual a dinâmica entre os setores que contam com profissionais de dados. Será que podemos encontrar setores com atuações mais similares entre si? Para abordar esse problema, analisaremos as competências dos funcionários desses setores.

Aqui, queremos identificar quais setores são próximos com base no uso de tecnologias. O grafico a seguirrepresenta a proximidade entre os setores, sendo que as cores mais "quentes" indicam maior similaridade. Vale destacar que consideramos apenas as áreas que foram ressaltadas na seção anterior, pois são aquelas que representam 80% da demanda por profissionais de dados.

In [54]:
def make_dummie_columns(df, columns, prefix):
    df_dummy = pd.concat([df, pd.get_dummies(df[columns], prefix=prefix, dummy_na=True)], axis=1)
    df_dummy = df_dummy.drop(columns, axis=1)
    return df_dummy

def build_similarities(df, agg_columns, skills_columns):
    dict_variable = {key: sum for key in skills_columns}
    df = df.groupby(agg_columns, as_index=False).agg(dict_variable)
    results = {}
    unique_groups = df.groupby(agg_columns).size().reset_index().drop(0, axis=1)
    data_list = []
    for group1, group2 in combinations(unique_groups.iterrows(), 2):
        group1_skills = df[df[agg_columns].eq(group1[1]).all(axis=1)].drop(agg_columns, axis=1).values
        group2_skills = df[df[agg_columns].eq(group2[1]).all(axis=1)].drop(agg_columns, axis=1).values
        cosine_similarity = np.dot(group1_skills[0], group2_skills[0]) / (np.linalg.norm(group1_skills[0]) * np.linalg.norm(group2_skills[0]))
        data_list.append({
            'group1': ''.join([x for x in group1[1]]),
            'group2': ''.join([x for x in group2[1]]),
            'similaridade': cosine_similarity
        })
    return pd.DataFrame(data_list)

def build_heatmap_with_distances(result_df, title):
    unique_groups = pd.unique(result_df[['group1', 'group2']].values.ravel())
    heatmap_data = []

    for group1 in unique_groups:
        row_data = []
        for group2 in unique_groups:
            if group1 == group2:
                row_data.append(1)
            else:
                distance = result_df.loc[
                    ((result_df['group1'] == group1) & (result_df['group2'] == group2)) |
                    ((result_df['group1'] == group2) & (result_df['group2'] == group1))
                ]['similaridade'].values
                row_data.append(distance[0] if len(distance) > 0 else 0)
        heatmap_data.append(row_data)

    fig = go.Figure(data=go.Heatmap(
        z=heatmap_data,
        x=unique_groups,
        y=unique_groups,
        colorscale='purp',
    ))

    fig.update_layout(
        title_text = title,
                title={
            'x': 0.5,
            'xanchor': 'center'
        },
        font_size=14,
        height=600,
        width=800,
        font_color="#42293f",
        font_family = "Graphic"

    )

    # Add opacity to values < 0.8 in the colorscale
    opacity = 0.5
    new_colorscale = []
    for scale in fig.data[0].colorscale:
        if scale[0] < 0.9:
            color = f'rgba{scale[1][3:-1]}, {opacity})'
            new_colorscale.append((scale[0], color))
        else:
            new_colorscale.append(scale)
    fig.data[0].colorscale = new_colorscale

    fig.show()


In [55]:
#Filtro setores foco
SoD22 = SoD22[SoD22["('P2_b ', 'Setor')"].isin(setores_foco)]
# alterando formato do df para uso por skills
SoD22_dummie = make_dummie_columns(SoD22, "('P1_l ', 'Nivel de Ensino')","('P1_l," )
SoD22_dummie = make_dummie_columns(
    SoD22_dummie, "('P4_e ', 'Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?')", "('P4_e,"
)

# criando lista de todos os skills que queremos usar
all_skills_columns = (
    get_list_columns("P4_c_", SoD22_dummie) +
    get_list_columns("P4_g_", SoD22_dummie) +
    get_list_columns("P4_f_", SoD22_dummie) +
    get_list_columns("P4_h_", SoD22_dummie) +
    get_list_columns("P4_i_", SoD22_dummie) +
    get_list_columns("P4_e", SoD22_dummie) +
    get_list_columns("P1_l", SoD22_dummie)
)

# df_similar_atuacao = build_similaridades(SoD22_dummie, [ "('P4_a_1 ', 'Atuacao')"], all_skills_columns)

df_similar_setor = build_similarities(SoD22_dummie, ["('P2_b ', 'Setor')" ], all_skills_columns )
build_heatmap_with_distances(df_similar_setor,'Similaridade entre Setores')

Insights a partir das similaridades entre setores:
1) Tecnologia/Fábrica de Software e Área de Consultoria apresentam uma alta similaridade, com um índice de similaridade de 0.983696. Isso indica que os profissionais de dados que atuam nesses setores possuem competências e habilidades semelhantes.

2) Setor de Finanças ou Bancos também possui uma forte afinidade com Tecnologia/Fábrica de Software, com um índice de similaridade de 0.977030. Isso sugere que os profissionais de dados nessas áreas compartilham competências relevantes para ambos os setores.

3) Por fim, o Setor de Internet/Ecommerce e o Setor de Energia apresentam uma das menores similaridades, com um índice de 0.805936. Isso sugere que os profissionais de dados que atuam nesses setores possuem habilidades menos semelhantes.

No entanto é necessário destacar que **todos os setores apresentaram altos índices de similaridade**. Ao considerar o agregado de todos os skills nas áreas, a alta similaridade entre os setores pode ser explicada pelo fato de que os profissionais de dados possuem competências e habilidades amplamente aplicáveis em diferentes áreas.  Podemos concluir que os profissionais de dados têm **habilidades que transcendem setores específicos, o que indica a versatilidade e a relevância de suas competências em toda a indústria**.

# 2. Competências Essenciais para Profissionais de Dados

Agora que temos uma visão geral do cenário atual do mercado de dados, é hora de nos aprofundarmos nas competências e habilidades que impulsionam o valor que geramos. Vamos explorar **quais skills são mais valorizados e essenciais** para os profissionais de dados, buscando insights que possam guiar os profissionais em suas jornadas de carreira.

O gráfico a seguir é a representação de todos os profissionais indicando sua posição com relação ao seu salário e ao número de skills que estes responderam que utilizam.

Para esta pesquisa não tivemos o valor exato dos salários, mas sim em qual faixa salarial o respondente se encontrava. Por este motivo, esta análise considerou o valor médio de cada faixa, exceto os extremos inferior e superior no qual usamos os valores mínimo e máximo, respectivamente.

O gráfico também mostra o centroíde de cada grupo, que é o valor médio dos pontos de dados no grupo. O centroíde pode ser usado para representar o grupo como um todo e para comparar os grupos uns com os outros.

In [56]:
def create_scatter_nskill_wage(df_n_skills, centroid_df):
    fig = px.scatter(df_n_skills,
                     x="n_skills",
                     y="salario",
                     color="Atuacao",
                     color_discrete_sequence=[ "#EB12AA", "#6A97F5", color_top[1]],
                     opacity=.5,
                     marginal_x="histogram",
                    )
    symbols = ["square", "star", 'diamond']
    for i, atuacao in enumerate(centroid_df['Atuacao']):
        df_temp = centroid_df[centroid_df['Atuacao']==atuacao].reset_index(drop=True)
        fig.add_trace(go.Scatter(
                            x = df_temp['n_skills'],
                            y = df_temp['salario'],
                            name = atuacao + " centroid",
                            hoverinfo='text',
                            hovertext=f"Area: {atuacao}<br>Salario: R${df_temp['salario'][0]}<br>n_skills: {df_temp['n_skills'][0]}" ,
                            hoverlabel_font_color = "white",
                            mode='markers',
                            marker=dict(
                                symbol= symbols[i],
                                    color = [ "#EB12AA", "#6A97F5", color_top[1]][i],
                                    line_color='white',
                                    line_width=.5,
                                    size=12,
                                    opacity = opacity_top
                                    ),
                            showlegend=True),
                      row=1,
                      col=1)

    fig.update_layout(
            title_text = 'Relação entre SKills e Faixa Salarial',
            title={
                'x': 0.5,
                'xanchor': 'center'
            },
            font_size=14,
            font_color="#42293f",
            font_family = "Graphic",
            xaxis=dict(
                    showgrid=True,
                    gridcolor='#e6e6dc',
                    gridwidth = .05,
                    tickfont_color= "#42293f",
                    showticklabels=True,
                    ticks='outside',
                    tickcolor="#42293f",
                    ),
            yaxis=dict(
                    showgrid=True,
                    ),
            width=800,
            height=600,
            paper_bgcolor='white',
            plot_bgcolor='white',
            showlegend=True,
#              legend=dict(
#                 orientation="h",
#                 yanchor="bottom",
#                 y=1.02,
#                 xanchor="right",
#                 x=1
#             )
            )
    # fig.update_traces(orientation='h')
    fig.show()

In [57]:
# selecao de todas habilidades em comum que as pessoas usaram em algum momento de suas vidas
all_skills_used_columns = (get_list_columns("P4_b_", SoD22_dummie)+
                            get_list_columns("P4_d_", SoD22_dummie) +
                            get_list_columns("P4_g_", SoD22_dummie)+
                            get_list_columns("P4_f_", SoD22_dummie)+
                            get_list_columns("P4_h_", SoD22_dummie) +
                            get_list_columns("P4_i_", SoD22_dummie)
                            )
dict_variable = {key:sum for key in all_skills_used_columns}
df_n_skills = SoD22_dummie[
    ["('P0', 'id')",
     "('P4_a_1 ', 'Atuacao')",
     "('P2_b ', 'Setor')"]+
    get_list_columns('P2_h',SoD22_dummie)
    +all_skills_used_columns].copy()
df_n_skills = df_n_skills.fillna(0)
df_n_skills['n_skills'] = df_n_skills[all_skills_used_columns].sum(axis=1)
df_n_skills = df_n_skills.rename(columns=column_renaming)


# aplicando o valor medio da faixa para analise quantitativa,
# exceto 1 e ultimo intervalos
dc_salario = {'de R$ 8.001/mês a R$ 12.000/mês': 10000,
'de R$ 12.001/mês a R$ 16.000/mês':14000,
'de R$ 4.001/mês a R$ 6.000/mês':5000,
'de R$ 6.001/mês a R$ 8.000/mês':7000,
'de R$ 3.001/mês a R$ 4.000/mês': 3500,
'de R$ 2.001/mês a R$ 3.000/mês':2500,
'de R$ 1.001/mês a R$ 2.000/mês':1500,
'de R$ 25.001/mês a R$ 30.000/mês':27500,
'de R$ 20.001/mês a R$ 25.000/mês':22500,
'de R$ 16.001/mês a R$ 20.000/mês':18000,
'de R$ 30.001/mês a R$ 40.000/mês':35000,
'Acima de R$ 40.001/mês':40000,
'Menos de R$ 1.000/mês':1000}

df_n_skills['salario'] = df_n_skills['Faixa salarial'].apply(lambda x: dc_salario[x])

# encontrando os centroides por atuacao
centroid_df = df_n_skills.groupby('Atuacao')[['n_skills', 'salario']].mean()
centroid_df = centroid_df.reset_index()

#plot
create_scatter_nskill_wage(df_n_skills, centroid_df)

Analisando o gráfico podemos notar alguns insights:

* **Engenharia de Dados é a área com maior número de skills**: Os profissionais de Engenharia de Dados possuem uma ampla variedade de competências, com algumas amostras apresentando até 21 skills diferentes. Isso pode indicar a natureza abrangente e técnica dessa área de atuação, exigindo dos profissionais uma procura contínua por novas tecnologias.

*   **Análise de Dados exige habilidades mais concentradas**: A área de Análise de Dados apresenta um número menor de skills, variando de 2 a 10. Isso pode indicar que os profissionais de análise de dados se concentram em competências específicas e essenciais para a visualização e interpretação de dados.

*   **Ciência de Dados também requer um conjunto diversificado de habilidades**: Profissionais que atuam na área de Ciência de Dados apresentam um elevado número de skills, variando de 6 a 13. Isso reflete a necessidade de conhecimentos intermediário entre as áreas de  análise  e engenharia de dados.

*    Salários variam de acordo com as habilidades e a área de atuação: Observa-se uma variação significativa nos salários entre as diferentes amostras de cada área. Isso sugere que profissionais com habilidades mais especializadas e avançadas tendem a receber salários mais altos. Um maior conhecimento em diversas tecnologias não significa necessáriamente maiores salários.
* Em média **engenharia e ciência de dados apresentam as maiores médias salariais**.


Aqui, iniciamos o estudo das competências que cada setor utiliza. Neste primeiro gráfico, queremos entender uma visão geral dessas habilidades pelos setores. As próximas análises dessa seção vão considerar valores percentuais relativos a cada setor no lugar dos valores absolutos de participantes. Isto é utilzado para auxiliar nas comparações entre setores e áreas de atuação.



In [58]:
color_seq = [color_top[0],color_top[3]  ] + ["#dadada" for x in range(90)]

def create_agg_sum_dict(skills_columns):
    return {key:sum for key in skills_columns}

def create_skilled_df(df, df_setor_all, agg_key):

    df_new = df.set_index(agg_key).stack().reset_index()
    df_new.columns = agg_key + ["skill", "Count_old"]
    df_new = df_new.merge(df_setor_all, on="Setor", how="left")
    df_new['Count'] =df_new.apply(lambda x: x['Count_old']/x['TotalSetor']*100, axis=1)
    df_new = df_new.sort_values(["Setor","Count" ], ascending=False)
    duplicated_rows = []

    # adiciona o primeiro elemento ao fim para cada combinação -- usado p "fechar o grafico"
    seen_combinations = set()

    for index, row in df_new.iterrows():
        combination = (row['Setor'], row['skill'])
        if combination not in seen_combinations:
            seen_combinations.add(combination)
            duplicated_rows.append(row)  # adiciona a 1 linha daquela comb caso nao exista

    return pd.concat([df_new, pd.DataFrame(duplicated_rows)] )



def create_single_polar(df_new, fig, row, column):
    """Esta função existe para criar apenas 1 grafico radial observando a distribuição entre os skills"""

    top2 = df_new.skill.unique()[:2]
    is_first = "Atuacao" not in df_new.columns
    for i, skill in enumerate(df_new.skill.unique()):
        df_skill = df_new[df_new['skill'] == skill]

        fig.add_trace(        go.Scatterpolar(
                    r=df_skill['Count'],
                    theta=df_skill['Setor'],
                    name=skill,
                    line_color=color_seq[i],
                    line_width=2 if skill in top2 else .5,
                    line_shape='spline',
                    line_smoothing=.5,
                    legendgroup=skill,
                    showlegend = True if (skill in top2) and is_first else False ,
                     hoverlabel_font_color = "white" if skill in top2 else"black",
                    ),
              row=row,
              col=column)
    fig.update_layout(
        title_text = "SQL e Planilhas fortes em todos os setores",
        title={
                'x':0.5,
                'xanchor': 'center'},
        font_size=12,
         font_color="#42293f",
        polar_bgcolor='white',
        polar_radialaxis_visible=True,
        polar_radialaxis_showticklabels=True,
        polar_radialaxis_tickfont_color='darkgrey',
        polar_angularaxis_color='grey',
        polar_angularaxis_showline=True,
        polar_radialaxis_showline=False,
#         polar_radialaxis_layer='below traces',
        polar_radialaxis_gridcolor='#dadada',
        polar_radialaxis_range=(0,125),
        polar_radialaxis_tickvals=[25, 50, 75, 100],
        polar_radialaxis_ticktext=['25%', '50%', "75%", "100%"],
        polar_radialaxis_tickmode='array',
         width=900,
            height=1200,

    )

def create_df_agg_by_key(SoD22_dummie_all, agg_key ):
    dict_variable = create_agg_sum_dict(SoD22_dummie_all.columns[3:])
    df_sector_skills = SoD22_dummie_all.groupby(agg_key, as_index=False).agg(dict_variable)
    return create_skilled_df(df_sector_skills, df_setor_all, agg_key)

def create_top_atuacao_table(SoD22_dummie_formacao_atuacao,fig , row = 1, column=2):
    df = (SoD22_dummie_formacao_atuacao[["Setor", "Atuacao", "skill", "Count"]]
          .drop_duplicates(keep="first")
          .rename(columns={"Count":'%'})
          .sort_values("%", ascending =  False ).head(10)
    )
    fig.add_trace(go.Table(header=dict(values=df.columns,
                                        line_color='darkslategray',
                                        fill_color=color_top[1],
                                        align=['center']*len(df.columns),
                                        font=dict(color='white', size=12)
                                       ),
                     cells=dict(values=[df["Setor"],
                                        df["Atuacao"],
                                        df["skill"],
                                        round(df["%"],0)
                                       ],
                            line_color='darkslategray',
                            fill_color = [[color_top[3],color_top[2]]*len(df)],
                            align = ['center']*len(df.columns),
                            font = dict(color = 'white', size = 10)
                               ),
                           columnwidth = [650,600,800,200],
                           ),
                  row=row,
                   col=column)

In [59]:
# selecao de todas habilidades em comum

dict_variable = {key:sum for key in all_skills_columns}
SoD22_dummie_all =SoD22_dummie[["('P0', 'id')","('P4_a_1 ', 'Atuacao')","('P2_b ', 'Setor')"]+all_skills_columns].rename(columns=column_renaming)

# cria df agrupando pela chave
agg_key = ["Setor"]
dict_variable = create_agg_sum_dict(SoD22_dummie_all.columns[3:])
df_sector_skills = SoD22_dummie_all.groupby(agg_key, as_index=False).agg(dict_variable)
df_new = create_skilled_df(df_sector_skills, df_setor_all,agg_key)

# plot todos skills
fig = make_subplots(rows=1,
                    cols=1,
                    horizontal_spacing=0,
                    specs=[[{'type': 'polar'}]])

create_single_polar(df_new, fig, 1, 1)

fig.update_layout(
    title_text = "SQL e Planilhas fortes em todos os setores",
    title={
        'x': 0.5,
        'xanchor': 'center'
    },
    font_size=14,
    font_color="#42293f",
    font_family = "Graphic",
    polar_bgcolor='white',
    polar_radialaxis_visible=True,
    polar_radialaxis_showticklabels=True,
    polar_radialaxis_tickfont_color='#dadada',
    polar_angularaxis_color='grey',
    polar_angularaxis_showline=True,
    polar_radialaxis_showline=False,
    polar_radialaxis_layer='below traces',
    polar_radialaxis_gridcolor='#dadada',
    polar_radialaxis_range=(0,125),
    polar_radialaxis_tickvals=[25, 50, 75, 100],
    polar_radialaxis_ticktext=['25%', '50%', "75%", "100%"],
    polar_radialaxis_tickmode='array',
    height = 400,
    width=950,


)
fig.show()

O **SQL e as planilhas** são as habilidades mais importantes para todos os setores analisados. Essa constatação sugere uma alta presença de fontes estruturadas de dados e uma preferência pelo uso tradicional de planilhas nas empresas para abordar as questões relacionadas aos dados.

A seguir, destacaremos as habilidades de acordo com algumas categorias e levando em consideração as áreas técnicas (análise, ciência e engenharia de dados) para entender como esses skills estão distribuídos.

# 2.1. Nível de formação
 Como está o nível de formação destes profissionais por área?

In [60]:
def create_bar_skills(SoD22_dummie_formacao, fig1):
    # barplot para atuacao nos top 3 setore
    df_undummie = SoD22_dummie_formacao.set_index(['Atuacao',"id", "Setor"]).stack().reset_index().rename(columns={"level_3": 'skill',                                                                                                              0:"valor"})
    df_undummie_skill_atuacao = df_undummie.groupby(["skill", 'Atuacao'], as_index=False).agg({"valor": sum})
    df_undummie_skill = df_undummie.groupby(["skill"], as_index=False).agg({"valor": sum}).rename(columns={"valor": "valor_total"})

    top_3_skills = df_undummie_skill.sort_values('valor_total', ascending=False).head(3).skill.unique()
    df = df_undummie_skill_atuacao[df_undummie_skill_atuacao.skill.isin(top_3_skills)]
    i=0
    for atuacao, group in df.groupby('Atuacao'):

        fig1.add_trace(go.Bar(
            y=group['skill'],
            x=group['valor'],
            name=atuacao,
            orientation='h',
            marker_color=color_top[i],

        ), col=2, row=1)
        i+=1

In [61]:
# selecao de todas habilidades em comum
skills_columns = get_list_columns("P1_l,", SoD22_dummie)

SoD22_dummie_formacao =SoD22_dummie[["('P0', 'id')","('P4_a_1 ', 'Atuacao')","('P2_b ', 'Setor')"]+skills_columns]


SoD22_dummie_formacao =SoD22_dummie[["('P0', 'id')","('P4_a_1 ', 'Atuacao')","('P2_b ', 'Setor')"]+skills_columns]
new_column_renaming= {col: col.split(",")[1][1:].replace("'",'').strip() for col in SoD22_dummie_formacao.columns}
SoD22_dummie_formacao=SoD22_dummie_formacao.rename(columns=column_renaming).rename(columns=new_column_renaming)


SoD22_dummie_formacao_setor =  create_df_agg_by_key(SoD22_dummie_formacao, ["Setor"] )
# SoD22_dummie_formacao_atuacao =  create_df_agg_by_key(SoD22_dummie_formacao, ["Setor", "Atuacao"] )


fig1 = make_subplots(rows=1,
                    cols=2,
                horizontal_spacing=.3,
                    specs=[[{'type': 'polar'},{'type': 'bar'}]])

create_single_polar(SoD22_dummie_formacao_setor, fig1, 1, 1)



create_bar_skills(SoD22_dummie_formacao,fig1)
# create_top_atuacao_table(SoD22_dummie_formacao_atuacao, fig=fig1)

fig1.update_layout(
    title_text = "SQL e Planilhas fortes em todos os setores",
        title={
            'x': 0.5,
            'xanchor': 'center'
        },
        font_size=14,

        font_color="#42293f",
        font_family = "Graphic",
    polar_bgcolor='white',
    polar_radialaxis_visible=True,
    polar_radialaxis_showticklabels=True,
    polar_radialaxis_tickfont_color='darkgrey',
    polar_angularaxis_color='grey',
    polar_angularaxis_showline=True,
    polar_radialaxis_showline=False,
    polar_radialaxis_layer='below traces',
    polar_radialaxis_gridcolor='#dadada',
    polar_radialaxis_range=(0,60),
    polar_radialaxis_tickvals=[25, 50],
    polar_radialaxis_ticktext=['25%', '50%', "75%", "100%"],
    polar_radialaxis_tickmode='array',
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
    ),
    paper_bgcolor='white',
    plot_bgcolor='white',
        xaxis=dict(
            showgrid=True,
            gridcolor='#e6e6dc',
            gridwidth=0.05,
            tickfont_color="#42293f",
            showticklabels=True,
            dtick=100,
            ticks='outside',
            tickcolor="#42293f"
        ),
    width=1050,
    height=550,

)
fig1.show()


Ao analisar os percentuais relativos ao nível de ensino para os cargos podemos obter alguns insights. Primeiramente, fica evidente que a categoria de **Graduação/Bacharelado é a mais representativa em todas as áreas**, com percentuais significativos de 41.73% para Analistas de Dados, 34.14% para Cientistas de Dados e 40.96% para Engenheiros de Dados. Isso indica que a grande maioria dos profissionais nessas áreas possui uma formação de graduação. Além disso existe uma participação dos três grupos em estudantes de graduação, fortalecendo a indicação de busca pela formação acadêmica dos profissionais.

Além disso, observamos que os Cientistas de Dados se destacam com o maior nível de educação entre os três grupos. A categoria tem uma representação grande com Doutorado ou PhD e Mestrado, demonstrando que eles contam com uma parcela considerável de profissionais altamente qualificados, com níveis avançados de educação.

Outra constatação relevante é que a **Pós-graduação tem uma presença marcante em todas as áreas**. Tanto Analistas de Dados (31.10%) quanto Engenheiros de Dados (33.99%) e Cientistas de Dados (19.59%) exibem proporções significativas de profissionais que buscam uma formação educacional mais avançada. Essa tendência sugere que muitos profissionais tem investido na especialização para aprimorar suas habilidades e conhecimentos técnicos.

Esses insights destacam as diferenças e similaridades nos níveis de educação entre os três cargos e podem ser de extrema utilidade no planejamento do desenvolvimento profissional do leitor. Além disso, essa análise enfatiza a importância crucial da educação e especialização contínua para que os profissionais se destaquem no mercado de dados. Investir em conhecimento e aprimoramento técnico é essencial para impulsionar o crescimento e o sucesso profissional.

> "Busquem conhecimento." - Bilu, Et.


# 2.2 Linguagem de programação
Acredito que a pergunta mais frequente que os interessados em adentrar na carreira de dados fazem é que linguagem deveriam começar a estudar. Aqui iremos explorar quais as linguagens mais utilizadas pelos profissionais em seu dia a dia.


In [62]:
# selecao de todas habilidades em comum
skills_columns = get_list_columns("P4_e", SoD22_dummie)

SoD22_dummie_ling =SoD22_dummie[["('P0', 'id')","('P4_a_1 ', 'Atuacao')","('P2_b ', 'Setor')"]+skills_columns]

new_column_renaming= {col: col.split(",")[1][1:].replace("'",'').strip() for col in SoD22_dummie_ling.columns}
SoD22_dummie_ling=SoD22_dummie_ling.rename(columns=column_renaming).rename(columns=new_column_renaming)

SoD22_dummie_ling_setor =  create_df_agg_by_key(SoD22_dummie_ling, ["Setor"] )
SoD22_dummie_ling_atuacao =  create_df_agg_by_key(SoD22_dummie_ling, ["Setor", "Atuacao"] )


fig2 = make_subplots(rows=1,
                    cols=2,
                horizontal_spacing=.2,
                    specs=[[{'type': 'polar'},{'type': 'bar'}]])

create_single_polar(SoD22_dummie_ling_setor, fig2, 1, 1)
# create_top_atuacao_table(SoD22_dummie_ling_atuacao, fig=fig2)
create_bar_skills(SoD22_dummie_ling,fig2)

fig2.update_layout(
    title_text = "Python e SQL lideram",
        title={
            'x': 0.5,
            'xanchor': 'center'
        },
        font_size=14,

        font_color="#42293f",
        font_family = "Graphic",
    polar_bgcolor='white',
    polar_radialaxis_visible=True,
    polar_radialaxis_showticklabels=True,
    polar_radialaxis_tickfont_color='darkgrey',
    polar_angularaxis_color='grey',
    polar_angularaxis_showline=True,
    polar_radialaxis_showline=False,
    polar_radialaxis_layer='below traces',
    polar_radialaxis_gridcolor='#dadada',
    polar_radialaxis_range=(0,60),
    polar_radialaxis_tickvals=[25, 50],
    polar_radialaxis_ticktext=['25%', '50%', "75%", "100%"],
    polar_radialaxis_tickmode='array',
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
    ),
    paper_bgcolor='white',
    plot_bgcolor='white',
        xaxis=dict(
            showgrid=True,
            gridcolor='#e6e6dc',
            gridwidth=0.05,
            tickfont_color="#42293f",
            showticklabels=True,
            dtick=100,
            ticks='outside',
            tickcolor="#42293f"
        ),
    width=1050,
    height=600,
)
fig2.show()

Alguns insights que podemos tirar da figura:

*    **Python é lidera nas áreas de Consultoria, finanças** e setor energético. Python é uma linguagem de programação amplamente utilizada em análise e ciência de dados, o que indica sua relevância nesse setor.

 * **Destaque para SQL**: Notamos que o SQL é a linguagem de programação mais amplamente utilizada na maioria dos setores, representando uma porcentagem significativa entre os profissionais de dados. SQL é essencial para acessar e manipular dados em bancos de dados, e sua presença destaca a importância de fontes estruturadas de dados em todos setores.

*   Níveis mais baixos de programação: Em geral, observamos que as linguagens de programação menos comuns são C/C++/C#, Java, JavaScript, .NET, PHP, SAS/Stata, Scala e Visual Basic/VBA. Elas estão presentes em menos de 5% das competências em todos os setores, sugerindo que essas linguagens podem não ser tão predominantes nas atividades, mas existem em aplicações específicas do setor.

* **Python emerge como uma das linguagens essenciais para Cientistas e Engenheiros de Dados.** Tanto Cientistas quanto Engenheiros apresentam percentuais elevados de profissionais com habilidades em Python, com 96.46% e 94.12%, respectivamente.

* Observamos que a linguagem R é mais comum entre os Cientistas de Dados em comparação com Analistas e Engenheiros de Dados. Cerca de 24.81% dos Cientistas de Dados possuem habilidades em R, enquanto Analistas e Engenheiros de Dados apresentam 15.71% e 4.36%, respectivamente. Essa diferença pode refletir a existência de uma preferência ou rixa dentro da comunidade de dados entre as linguagens R e Python. No entanto, é importante notar que, de forma geral, Python ainda é a linguagem mais predominante nas competências, liderando o conjunto de habilidades técnicas em análise e ciência de dados.

* Outro ponto relevante é que a linguagem Java, assim como JavaScript e Scala, é amplamente utilizada pelos Engenheiros de Dados. Essas linguagens representam 11.98% das habilidades dos Engenheiros, enquanto para Analistas de Dados e Cientistas de Dados, os valores são de 4.91% e 5.41%, respectivamente.

Esses insights evidenciam as diferenças nas habilidades e conhecimentos em diferentes linguagens de programação entre os três cargos de dados, o que pode auxiliar na identificação de lacunas de habilidades e na formulação de estratégias de desenvolvimento profissional mais específicas para cada área de atuação. Embora o SQL e Python sejam essenciais em todas as três áreas, cada cargo possui suas particularidades em relação a outras linguagens, ressaltando a importância de uma abordagem personalizada para a capacitação de profissionais em cada campo de dados. Conhecimentos diversificados e especializados em linguagens são fundamentais para se destacar em um mercado de dados em constante evolução.

# 2.3 Fontes de dados

Outro dado importante que a pesquisa nos forneceu foi a indicação das fontes de dados com as quais os profissionais têm contato em seu trabalho. A seguir, destacamos as fontes de dados mais comuns às quais os profissionais têm acesso e as fontes de dados que eles utilizam com mais frequência.

In [63]:
# selecao de todas habilidades em comum
skills_columns = get_list_columns("P4_c_", SoD22_dummie)

SoD22_dummie_fd =SoD22_dummie[["('P0', 'id')","('P4_a_1 ', 'Atuacao')","('P2_b ', 'Setor')"]+skills_columns]

SoD22_dummie_fd=SoD22_dummie_fd.rename(columns=column_renaming)

SoD22_dummie_fd_setor =  create_df_agg_by_key(SoD22_dummie_fd, ["Setor"] )
SoD22_dummie_fd_atuacao =  create_df_agg_by_key(SoD22_dummie_fd, ["Setor", "Atuacao"] )


fig2 = make_subplots(rows=1,
                    cols=2,
                horizontal_spacing=.1,
                    specs=[[{'type': 'polar'},{'type': 'bar'}]])

create_single_polar(SoD22_dummie_fd_setor, fig2, 1, 1)
# create_top_atuacao_table(SoD22_dummie_fd_atuacao, fig=fig2)
create_bar_skills(SoD22_dummie_fd, fig2)

fig2.update_layout(
    title_text = "Dados Relacionais e Planilhas por todos os lados",
        title={
            'x': 0.5,
            'xanchor': 'center'
        },
        font_size=14,

        font_color="#42293f",
        font_family = "Graphic",
    polar_bgcolor='white',
    polar_radialaxis_visible=True,
    polar_radialaxis_showticklabels=True,
    polar_radialaxis_tickfont_color='darkgrey',
    polar_angularaxis_color='grey',
    polar_angularaxis_showline=True,
    polar_radialaxis_showline=False,
    polar_radialaxis_layer='below traces',
    polar_radialaxis_gridcolor='#dadada',
    polar_radialaxis_range=(0,110),
    polar_radialaxis_tickvals=[25, 50, 75,100],
    polar_radialaxis_ticktext=['25%', '50%', "75%", "100%"],
    polar_radialaxis_tickmode='array',
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=.95,
    xanchor="left",
    x=0
    ),
    paper_bgcolor='white',
    plot_bgcolor='white',
        xaxis=dict(
            showgrid=True,
            gridcolor='#e6e6dc',
            gridwidth=0.05,
            tickfont_color="#42293f",
            showticklabels=True,
            dtick=100,
            ticks='outside',
            tickcolor="#42293f"
        ),
    width=1000,
    height=600,

)
fig2.show()

* **Importância de Fontes de Dados Relacionais e Planilhas**: Dados relacionais estruturados em bancos SQL e planilhas são as fontes de dados mais comuns e mais utilizadas pelos profissionais em todos os setores analisados. Isso ressalta a relevância de aprender sobre essas fontes de dados, pois elas são fundamentais para diversas atividades profissionais, independentemente do setor.

* Baixa Utilização de Imagens, Vídeos e Áudios: Embora os profissionais tenham contato em algum momento com fontes de dados de imagens, vídeos e áudios, essas fontes não são tão frequentemente utilizadas no dia a dia. Isso pode ser atribuído a uma maior ênfase em dados textuais e estruturados em suas atividades diárias.


# 2.4 Banco de dados
Vamos analisar o uso de bancos de dados pelos profissionais, indicando os bancos de dados mais utilizados por setor:

In [64]:
# selecao de todas habilidades em comum
skills_columns = get_list_columns("P4_g_", SoD22_dummie) +get_list_columns("P4_f_", SoD22_dummie)


SoD22_dummie_db =SoD22_dummie[["('P0', 'id')","('P4_a_1 ', 'Atuacao')","('P2_b ', 'Setor')"]+skills_columns]

SoD22_dummie_db=SoD22_dummie_db.rename(columns=column_renaming)

SoD22_dummie_db_setor =  create_df_agg_by_key(SoD22_dummie_db, ["Setor"] )
SoD22_dummie_db_atuacao =  create_df_agg_by_key(SoD22_dummie_db, ["Setor", "Atuacao"] )


fig2 = make_subplots(rows=1,
                    cols=2,
                horizontal_spacing=.1,
                    specs=[[{'type': 'polar'},{'type': 'bar'}]])

create_single_polar(SoD22_dummie_db_setor, fig2, 1, 1)
# create_top_atuacao_table(SoD22_dummie_db_atuacao, fig=fig2)
create_bar_skills(SoD22_dummie_db, fig2)

fig2.update_layout(
    title_text = "Diversidade nos bancos de dados ",
           title={
            'x': 0.5,
            'xanchor': 'center'
        },
        font_size=14,

        font_color="#42293f",
        font_family = "Graphic",
    polar_bgcolor='white',
    polar_radialaxis_visible=True,
    polar_radialaxis_showticklabels=True,
    polar_radialaxis_tickfont_color='darkgrey',
    polar_angularaxis_color='grey',
    polar_angularaxis_showline=True,
    polar_radialaxis_showline=False,
    polar_radialaxis_layer='below traces',
    polar_radialaxis_gridcolor='#dadada',
    polar_radialaxis_range=(0,60),
    polar_radialaxis_tickvals=[25, 50],
    polar_radialaxis_ticktext=['25%', '50%', "75%", "100%"],
    polar_radialaxis_tickmode='array',
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
    ),
    paper_bgcolor='white',
    plot_bgcolor='white',
        xaxis=dict(
            showgrid=True,
            gridcolor='#e6e6dc',
            gridwidth=0.05,
            tickfont_color="#42293f",
            showticklabels=True,
            dtick=100,
            ticks='outside',
            tickcolor="#42293f"
        ),
    width=1050,
    height=400,

)
fig2.show()

Ao analisarmos os dados, percebemos que, diferentemente de outras tecnologias em que uma ou duas opções dominam claramente, nos bancos de dados a diferença entre as fontes utilizadas é menor. Isso sugere que cada setor possui preferências distintas em relação às fontes de dados que utilizam. Em outras palavras, não existe uma fonte de dados única ou predominante que seja amplamente adotada por todos os setores. Em vez disso, as escolhas variam de acordo com as necessidades e características específicas de cada setor. Essa **diversidade na utilização de fontes de dados** reflete a natureza versátil e adaptável dos profissionais de cada área, que selecionam as fontes que melhor atendem às suas demandas e requisitos particulares.

* Os destaques que podemos levantar aqui são **SQL SERVER, MySQL, Google BigQuery e PostgreSQL**

# 2.5 Cloud

Agora, iremos explorar o uso de Cloud pelos profissionais utilizados em cada setor:


In [65]:
# selecao de todas habilidades em comum
skills_columns = get_list_columns("P4_h_", SoD22_dummie)

SoD22_dummie_cld =SoD22_dummie[["('P0', 'id')","('P4_a_1 ', 'Atuacao')","('P2_b ', 'Setor')"]+skills_columns]

SoD22_dummie_cld=SoD22_dummie_cld.rename(columns=column_renaming)

SoD22_dummie_cld_setor =  create_df_agg_by_key(SoD22_dummie_cld, ["Setor"] )
SoD22_dummie_cld_atuacao =  create_df_agg_by_key(SoD22_dummie_cld, ["Setor", "Atuacao"] )


fig2 = make_subplots(rows=1,
                    cols=2,
                horizontal_spacing=.15,
                    specs=[[{'type': 'polar'},{'type': 'bar'}]])

create_single_polar(SoD22_dummie_cld_setor, fig2, 1, 1)
# create_top_atuacao_table(SoD22_dummie_cld_atuacao, fig=fig2)
create_bar_skills(SoD22_dummie_cld, fig2)


fig2.update_layout(
    title_text = "Principais nuvens em uso",
        title={
            'x': 0.5,
            'xanchor': 'center'
        },
        font_size=14,

        font_color="#42293f",
        font_family = "Graphic",
    polar_bgcolor='white',
    polar_radialaxis_visible=True,
    polar_radialaxis_showticklabels=True,
    polar_radialaxis_tickfont_color='darkgrey',
    polar_angularaxis_color='grey',
    polar_angularaxis_showline=True,
    polar_radialaxis_showline=False,
    polar_radialaxis_layer='below traces',
    polar_radialaxis_gridcolor='#dadada',
    polar_radialaxis_range=(0,60),
    polar_radialaxis_tickvals=[25, 50],
    polar_radialaxis_ticktext=['25%', '50%', "75%", "100%"],
    polar_radialaxis_tickmode='array',
legend=dict(
    orientation="h",
    yanchor="bottom",
    y=.95,
    xanchor="left",
    x=0
    ),
    paper_bgcolor='white',
    plot_bgcolor='white',
        xaxis=dict(
            showgrid=True,
            gridcolor='#e6e6dc',
            gridwidth=0.05,
            tickfont_color="#42293f",
            showticklabels=True,
            dtick=100,
            ticks='outside',
            tickcolor="#42293f"
        ),
    width=1050,
    height=400,
)
fig2.show()

A **nuvem mais utilizada pelos profissionais é a aws, seguida por Azure e GCP**.

É importantedestacar que a pesquisa se limitou a questionar o uso das três maiores plataformas de cloud disponíveis, ou seja, Azure (Microsoft), Amazon Web Services (AWS) e Google Cloud (GCP). Portanto, não dispomos de informações sobre o uso dos profissionais em relação a outras provedoras de cloud, como Alibaba, IBM e Oracle, por exemplo. A pesquisa abrangeu apenas as três principais provedoras, e detalhes sobre o uso de outras opções de nuvem não foram contemplados nesse levantamento.

# 2.6 Business Intelligence
Os profissionais de dados utilizam ferramentas de Business Intelligence? Vamos explorar agora o uso de ferramentas de visualização de dados (BIs) pelos profissionais da área.


In [66]:
# selecao de todas habilidades em comum
skills_columns = get_list_columns("P4_i_", SoD22_dummie)

SoD22_dummie_bi =SoD22_dummie[["('P0', 'id')","('P4_a_1 ', 'Atuacao')","('P2_b ', 'Setor')"]+skills_columns]

SoD22_dummie_bi=SoD22_dummie_bi.rename(columns=column_renaming)

SoD22_dummie_bi_setor =  create_df_agg_by_key(SoD22_dummie_bi, ["Setor"] )
SoD22_dummie_bi_atuacao =  create_df_agg_by_key(SoD22_dummie_bi, ["Setor", "Atuacao"] )


fig2 = make_subplots(rows=1,
                    cols=2,
                horizontal_spacing=.25,
                    specs=[[{'type': 'polar'},{'type': 'bar'}]])

create_single_polar(SoD22_dummie_bi_setor, fig2, 1, 1)
# create_top_atuacao_table(SoD22_dummie_bi_atuacao, fig=fig2)
create_bar_skills(SoD22_dummie_bi, fig2)

fig2.update_layout(
    title_text = "BIs com espaço para crescimento",
            title={
            'x': 0.5,
            'xanchor': 'center'
        },
        font_size=14,

        font_color="#42293f",
        font_family = "Graphic",
    polar_bgcolor='white',
    polar_radialaxis_visible=True,
    polar_radialaxis_showticklabels=True,
    polar_radialaxis_tickfont_color='darkgrey',
    polar_angularaxis_color='grey',
    polar_angularaxis_showline=True,
    polar_radialaxis_showline=False,
    polar_radialaxis_layer='below traces',
    polar_radialaxis_gridcolor='#dadada',
    polar_radialaxis_range=(0,15),
    polar_radialaxis_tickvals=[5,10,15],
    polar_radialaxis_ticktext=['5%', '10%', "15%", "100%"],
    polar_radialaxis_tickmode='array',
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=.95,
    xanchor="left",
    x=0
    ),
    paper_bgcolor='white',
    plot_bgcolor='white',
        xaxis=dict(
            showgrid=True,
            gridcolor='#e6e6dc',
            gridwidth=0.05,
            tickfont_color="#42293f",
            showticklabels=True,
            dtick=100,
            ticks='outside',
            tickcolor="#42293f"
        ),
    width=1050,
    height=500,

)
fig2.show()

Em diversas áreas, uma parcela significativa dos profissionais (entre 45% e 64%) relatou não usar nenhuma ferramenta de BI em seu trabalho. Isso pode indicar oportunidades para implementar soluções de BI nessas organizações, visando melhorar a análise de dados e a tomada de decisões. Os destaques de ferramentas de BI foi o **Microsoft PowerBI**.
# 3. Rede de Skills

Agora que dispomos de uma visão abrangente sobre como as habilidades estão sendo empregadas pelos profissionais de dados, podemos analisar se há alguma proximidade entre os skills comuns a todas as áreas. A seguir, apresentamos uma rede que identifica as relações entre essas habilidades. Observe que os círculos maiores representam os skills com mais conexões com outras habilidades adjacentes. Essa representação gráfica nos auxiliará a visualizar as interações e relevância dos diferentes skills no contexto geral.

In [67]:
SoD22_dummie_ling_setor['prefix'] = "ling"
SoD22_dummie_fd_setor['prefix'] = "fd"
SoD22_dummie_db_setor['prefix'] = "db"
SoD22_dummie_cld_setor['prefix'] = "cld"
SoD22_dummie_bi_setor['prefix'] = "bi"

df_skill_top = pd.concat([SoD22_dummie_ling_setor,
SoD22_dummie_fd_setor,
SoD22_dummie_db_setor,
SoD22_dummie_cld_setor,
SoD22_dummie_bi_setor])

grouped_df = df_skill_top.groupby(["prefix", "skill"], as_index=False).agg({"Count_old": sum})
sorted_df = grouped_df.sort_values(by=["prefix", "Count_old"], ascending=[True, False])
top_skills = sorted_df.groupby("prefix").head(5)

df_skill_top = pd.DataFrame(top_skills)


new_column_renaming= {col: col.split(",")[1][1:].replace("'",'').strip() for col in SoD22_dummie.columns}
df_net = SoD22_dummie.rename(columns = column_renaming).rename(columns = new_column_renaming)
df_net=df_net[["id"] + list(df_skill_top.skill.unique())]


df_net = df_net.set_index("id")
correlation_matrix = df_net.corr()


df_stack = correlation_matrix.stack().reset_index()
df_stack = df_stack.rename(columns={0:"value"})
df_stack = df_stack[(df_stack.value>.1)&(df_stack.level_0!=df_stack.level_1)]

for d in range(len(df_stack)):
    G = nx.from_pandas_edgelist(df_stack, 'level_0', 'level_1')

## def nodes info
node_degrees = G.degree()
node_sizes = [5+5 * degree for node, degree in G.degree()]

## def edges
df_stack = correlation_matrix.stack().reset_index()
df_stack = df_stack.rename(columns={0: "value"})
df_stack = df_stack[(df_stack.value > 0.1) & (df_stack.level_0 != df_stack.level_1)]

<ipython-input-67-93e6fea4ee04>:26: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [68]:
# cria graf
G = nx.from_pandas_edgelist(df_stack, 'level_0', 'level_1')

# selecao do layout desejado
pos = nx.kamada_kawai_layout(G )


edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=2, color='#dadada'),
    hoverinfo='none',
    mode='lines'
)

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += (x0, x1, None)
    edge_trace['y'] += (y0, y1, None)

# conexoes entre skills
node_trace = go.Scatter(
    x=[],
    y=[],
    text=[node for node in G.nodes()],
    mode='markers+text',
    textposition='middle center',
    hoverinfo='text',
    marker=dict(
        showscale=False,
        size=node_sizes,
        color= color_top[2],
        line_width=2,
        opacity=1
    )
)
# skills top
for node in G.nodes():
    x, y = pos[node]
    node_trace['x'] += (x,)
    node_trace['y'] += (y,)

# fig plotly
fig = go.Figure(data=[edge_trace, node_trace],

                layout=go.Layout(
                    title="Skill Network",
                    titlefont_size=20,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=0, l=0, r=0, t=60),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                ))


fig.update_layout(
      title={
            'x':0.5,
            'xanchor': 'center'},
    font_family="Arial",
    font_size=12,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
)



# hoverinfo
node_labels = [f"{node}: {val:.2f}" for node, val in zip(df_stack['level_0'], node_sizes)]
node_trace.hoverinfo = 'text'
node_trace.text = node_labels

fig.show()

As interconexões entre as habilidades e tecnologias mais procuradas pelos profissionais de dados foram representadas em uma rede. É importante lembrar que a combinação de habilidades deve ser adaptada ao contexto específico de cada empresa e pessoa, buscando sempre atender às necessidades do mercado em constante evolução.
A seguir os principais insights:

*    Microsoft PowerBI apresenta uma forte relação com Azure (Microsoft), o que é intuitivo visto que ambos são oferecidos pela mesma cloud. SQL Server está em sinergia com essas tecnologias para análise de dados.

*    Google Data Studio possui uma conexão significativa com Google Cloud (GCP) e Google BigQuery, destacando sua relevância na visualização e análise de dados na plataforma da Google.

*    Amazon Web Services (AWS) tem conexões relevantes com Tableau, S3, Python, evidenciando sua abrangência e uso diversificado no mercado de dados.

*    SQL Server possui conexões com Microsoft PowerBI, Tableau, Azure (Microsoft), MySQL, Dados relacionais (estruturados em bancos SQL) e SQL, mostrando sua presença tanto em ferramentas de BI quanto em bancos de dados relacionais.

* A linguagem de programação** Python apresenta conexões relevantes com diversas ferramentas de BI**, como Metabase, Amazon Web Services (AWS), PostgreSQL e S3, destacando sua utilidade em diversos contexto de análise.

* A linguagem de programação R possui associações com a ferramenta Microsoft PowerBI e também com dados georreferenciados, revelando seu uso em análises espaciais.


* As tecnologias relacionadas a **bancos de dados, possuem conexões fortes com várias ferramentas de BI**, reforçando sua importância na análise e visualização de dados.


# 4. Conclusao

À medida que o mercado de dados continua a crescer, os profissionais de dados devem se manter atualizados com habilidades essenciais para prosperar em suas carreiras. **SQL e Python são linguagens de programação fundamentais**, enquanto tecnologias relacionadas a fontes de dados, como **SQL e planilhas, permanecem cruciais**. Diversos setores, como **finanças, tecnologia/software, consultoria e varejo**, demandam a expertise de profissionais de dados.

A educação superior é predominante entre os profissionais de dados, com muitos buscando especialização e aprimoramento por meio de pós-graduação.

Os aspirantes a profissionais de dados devem concentrar-se em dominar as habilidades essenciais, enquanto os profissionais experientes devem continuar aprendendo e se adaptando para permanecer relevantes em uma indústria em rápida evolução.

Ao identificar e aprimorar habilidades e tecnologias essenciais, os profissionais podem prosperar em suas carreiras e ter um impacto significativo em suas respectivas indústrias. A aprendizagem contínua e a disposição para se adaptar serão fundamentais para o sucesso no cenário em constante mudança dos dados.

Por fim, a simplificação é a chave para navegar pelas complexidades do mundo dos dados. É essencial refletir sobre seus objetivos e habilidades para prosperar nessa jornada profissional


# 5. Bonus: Modelo para prever o que estudar

Imagine que você deseja ingressar na área de dados como engenheiro em uma empresa do setor de varejo. Com o auxílio desse modelo, é possível identificar quais habilidades são mais valorizadas nesse contexto específico. Dessa forma, você pode focar seus esforços de aprendizado nas habilidades mais relevantes, aumentando suas chances de sucesso no processo seletivo.

Por outro lado, se você está responsável pelo recrutamento de profissionais de dados para sua empresa, o modelo também pode ser uma ferramenta valiosa. Ao analisar a rede de habilidades, você pode montar descrições de vagas mais precisas e identificar os candidatos mais qualificados para a posição em aberto.

Nos próximos exemplos, apresentaremos como esse modelo pode ser aplicado em situações reais, oferecendo insights estratégicos e contribuindo para o alinhamento entre as necessidades do mercado e as escolhas dos profissionais e das empresas.

*ATENÇÃO*:  É importante ressaltar que o modelo apresentado não tem o objetivo de ditar ou direcionar de forma absoluta as escolhas e trajetórias dos profissionais. Trata-se de um modelo gerado a partir dos dados de análise coletados, que oferece insights sobre as habilidades demandadas pelo mercado. Cada pessoa é única, com habilidades, interesses e objetivos distintos. Portanto, é fundamental compreender que as decisões de carreira devem ser tomadas com base em uma avaliação individual e criteriosa, levando em consideração o contexto específico do trabalho que se pretende desenvolver.



In [69]:
def adapt_input_to_model(df):

    # transtormando em dummie os inputs
    for col in df.select_dtypes(include=['object']):
        df = pd.concat([df,
                               pd.get_dummies(df[col],
                                              prefix=col, dummy_na=True)], axis=1)
        df = df.drop(col, axis=1)
    ordered_columns = df.columns
    df= df.fillna(0)
    df = df.dropna()
    return df



# selecao de todos os skills em formato categorico
df_temp = SoD22[
["('P2_b ', 'Setor')",
 "('P2_g ', 'Nivel')",
 "('P4_a_1 ', 'Atuacao')",
 "('P2_h ', 'Faixa salarial')",
 "('P2_c ', 'Numero de Funcionarios')",
 "('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')",
]+[ "('P4_c ', 'Entre as fontes de dados listadas, quais você utiliza na maior parte do tempo?')",
 "('P4_e ', 'Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?')",
 "('P4_g ', 'Quais dos bancos de dados/fontes de dados listados abaixo você utiliza no trabalho?')",
 "('P4_g ', 'Quais das opções de Cloud listadas abaixo você utiliza no trabalho?')",
  "('P4_i ', 'Microsoft PowerBI')",
]].copy().dropna(subset = ["('P2_b ', 'Setor')",
 "('P2_g ', 'Nivel')",
 "('P4_a_1 ', 'Atuacao')",
 "('P2_h ', 'Faixa salarial')",
 "('P2_c ', 'Numero de Funcionarios')",
 "('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')"
], how="any")

# definição das variáveis para treino
df =df_temp[["('P2_b ', 'Setor')",
 "('P2_g ', 'Nivel')",
 "('P4_a_1 ', 'Atuacao')",
 "('P2_h ', 'Faixa salarial')",
 "('P2_c ', 'Numero de Funcionarios')",
 "('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')"]]

df = adapt_input_to_model(df)

# nossos alvos são quais skills queremos indicar
target_df = df_temp[[ "('P4_c ', 'Entre as fontes de dados listadas, quais você utiliza na maior parte do tempo?')",
 "('P4_e ', 'Entre as linguagens listadas abaixo, qual é a que você mais utiliza no trabalho?')",
 "('P4_g ', 'Quais dos bancos de dados/fontes de dados listados abaixo você utiliza no trabalho?')",
 "('P4_g ', 'Quais das opções de Cloud listadas abaixo você utiliza no trabalho?')",
  "('P4_i ', 'Microsoft PowerBI')"]]
# target_df[]= target_df[get_list_columns("P4_c_", SoD22_dummie)].rename(columns=column_renaming).idxmax(axis=1)
target_df = target_df.fillna("Nenhum")


In [70]:
""" Irei utilizar um modelo de Random Forest classifier porque estamos tratando de um
problema com multi-classes no meu objetivo, assim optei pelo presente modelo pela sua
capacidade de trabalhar com complexidade hierarquica entre as classes.
"""
# definição dos conjuntos para o modelo
X = df
y = target_df

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# model
classifier = RandomForestClassifier(n_estimators = 83, random_state=33)
multi_output_classifier = MultiOutputClassifier(classifier)
multi_output_classifier.fit(df, target_df)

# test
y_pred = multi_output_classifier.predict(X_test)

# metric
accuracies = []
for i, col in enumerate(target_df.columns):
    accuracy_i = accuracy_score(y_test[col], y_pred[:,i])
    accuracies.append(accuracy_i)
overall_accuracy = np.mean(accuracies)

print("Acurácia:", overall_accuracy)

# disclaimer: a acurácia foi melhor do que eu inicialmente imaginei.
#Aqui não realizarei melhorias, mas é possivel considerar se as classes estão balanceadas e realizar uma análise das
#features utilizadas. utilize as análises para se guiar no que pode contribuir para melhorar os resultados do modelo.

Acurácia: 0.8730679156908664


Exemplo: suponha que o leitor está querendo adentrar na área de ciência de dados e deseja trabalhar para um banco.

E seleciona os seguintes parametros:
* Setor : Financas
* Nivel: JR
* Atuacao: Ciencia de dados
* Faixa salarial: 'de R 4.001/mês a R 6.000/mês'
* Numero de Funcionarios: Acima de 3.000
* Quanto tempo de experiência na área de dados você tem? 'Não tenho experiência na área de dados'

In [71]:
df_ex = pd.DataFrame({
    "('P2_b ', 'Setor')":['Finanças ou Bancos'],
 "('P2_g ', 'Nivel')":['Júnior',],
 "('P4_a_1 ', 'Atuacao')":['Ciência de Dados'],
 "('P2_h ', 'Faixa salarial')":['de R$ 4.001/mês a R$ 6.000/mês'],
 "('P2_c ', 'Numero de Funcionarios')":["Acima de 3.000"],

    "('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')":['Não tenho experiência na área de dados']})
df_ex

,"('P2_b ', 'Setor')","('P2_g ', 'Nivel')","('P4_a_1 ', 'Atuacao')","('P2_h ', 'Faixa salarial')","('P2_c ', 'Numero de Funcionarios')","('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')"
0,Finanças ou Bancos,Júnior,Ciência de Dados,de R$ 4.001/mês a R$ 6.000/mês,Acima de 3.000,Não tenho experiência na área de dados


In [72]:

def predict_skills(input_df):
    df = adapt_input_to_model(input_df)
    missing_cols = set(X_train.columns) - set(df.columns)
    for col1 in missing_cols:
        df[col1] = 0
    df = df[X_train.columns]
    return pd.DataFrame(multi_output_classifier.predict(df),
                        columns =["Fonte de dados", "Linguagem", "Banco de dados", "Cloud", "BI"])

In [73]:
predict_skills(df_ex)

,Fonte de dados,Linguagem,Banco de dados,Cloud,BI
0,Planilhas,Python,"Splunk, Hive, MySQL",Amazon Web Services (AWS),Nenhum


Enquanto o modelo pode fornecer informações valiosas, é essencial lembrar que o aprendizado e a qualificação são processos contínuos. A busca por conhecimento e aprimoramento deve ser uma constante na vida profissional de qualquer indivíduo, independentemente das tendências do mercado. O leitor pode realizar suas interpretaçoes com os graficos iterativos e também pesquisar fontes externas sobre o que é imteressante focar seus esforços para se qualificar.  

Portanto, utilize o modelo como uma ferramenta auxiliar, mas tome decisões conscientes e embasadas em suas próprias metas, aspirações e aptidões. Lembre-se de que **modelos matemáticos e análises trazem resultados quantitativos, mas pessoas que tomam decisões.**

Se chegou até aqui, tire um momento para refletir e apreciar esta citação.

> Simplicidade, simplicidade, simplicidade! Tenha dois ou três afazeres e não cem ou mil; em vez de um milhão, conte meia dúzia… No meio desse mar agitado da vida civilizada há tantas nuvens, tempestades, areias movediças e mil e um itens a considerar, que o ser humano tem que se orientar - se ele não afundar e definitivamente acabar não fazendo sua parte - por uma técnica simples de previsão, além de ser um grande calculista para ter sucesso. Simplifique, simplifique.“    

— Henry David Thoreau, Walden
